In [1]:
import sys
sys.path.insert(1, 'Module')
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
from tkinter import *

start_timer = time.perf_counter()
def print_start(name):
    print(f'Start Simulation {name}')

"##### Start programm with run #####"

"##### Input Parameter #####"
output_excel_S = "Variante_2b_ohne_öl.xlsx"  #anpassen nach Daten einlesen !!einsetzen!! je nach Variante Lastgang anpassen unten Zeile 47!
Sim_LG = 1 #Bilanzierung des Lastgangs [ 0 = aus / 1 = ein ]
ELKW_Sim = 3 #anzahl elkw 3 , [ 0 = aus ]
WELKW_Sim = 0 #anzahl welkw , [ 0 = aus]
EPKW_Sim = 1 # [ 0 = aus / 1 = ein ]
Speicher = list(range(0, 300, 136)) # (min, max, schrittgrösse) [ max < schrittgrösse -> nur PV bilanzierung ]
# Eingabe der PV-Anlagen (0 oder 1)
dach_süd = 0
dach_ostwest = 1
fassade_süd = 1
fassade_west = 1
fassade_nord = 1
fassade_ost = 1
carport = 0
excel = True
plot = True

"##### Daten einlesen #####"
# Die Daten für Lastgang und PV-Anlage werden in einem Format von 15 Min Werten über ein ganzes Jahr eingelesen
Feiertage = ["01-01","01-02", "12-25", "12-26"] #Tage ohne Arbeitsbetrieb für Eigenverbrauch anstatt Peak-Shaving
# Daten Mobilität, Format für einlesen
r_LKW1 = pd.read_excel(r'Daten_Lupfig\Mobiltät_LKW_2022_Fahrdaten.xlsx', 0)
r_LKW2 = pd.read_excel(r'Daten_Lupfig\Mobiltät_LKW_2022_Fahrdaten.xlsx', 1)
r_LKW3 = pd.read_excel(r'Daten_Lupfig\Mobiltät_LKW_2022_Fahrdaten.xlsx', 2)
r_LG = pd.read_excel(r'Daten_Lupfig\Analyse_Lastgang22_Lupfig_Taracell_20230315.xlsx', 4) #Lastgang 2
LG_name = "Effizient"
####### !!einsetzen für r_LG!!: r_LG_Trend_Künten (var4), r_LG_Effizient (var5) oder r_LG_öl (Ref, var1,2,3) ######

"einlesen der PV-Daten mit Polysun simuliert oder Messdaten"
r_PV_F = pd.read_excel(r'Daten_Lupfig\Berechnung_und_PolysunOutput_20221219.xlsx', 2) #PV_Fassaden
r_PV_DS = pd.read_excel(r'Daten_Lupfig\Berechnung_und_PolysunOutput_20221219.xlsx', 3) #PV_Dach_Süd
r_PV_DOW = pd.read_excel(r'Daten_Lupfig\Berechnung_und_PolysunOutput_20221219.xlsx', 4) #PV_Dach_OstWest
r_PV_CP = pd.read_excel(r'Daten_Lupfig\Berechnung_und_PolysunOutput_20221219.xlsx', 5) #PV_Dach_carport
spalte_analyse = "ANALYSE_SPITZE"


"##### TARIFE #####"
# Faktor Tarifvariation Energieanteil Arbeitspreis: 1 = Standard
Tar_var = 1.15
# Faktor Tarifvariation Leistungspreis: 1 = Standard
Lei_var = 1.15
leistungsspitzenpreis = 7.5 * Lei_var # CHF/kW
hochtarif = (0.1079 * Tar_var +0.0204 + 0.0296)*1.077 # CHF/kWh
niedertarif = (0.0964 * Tar_var +0.014 + 0.0296)*1.077 #0.16927 * Tar_var # CHF/kWh
Referenzmarktpreis = 76.92 # Q1 2023
rueckspeisungstarif = ((Referenzmarktpreis/1000-0.003)* Tar_var) + 0.0200 # CHF/kWhN_BS
tarif_schnellladen = 0.5 * Tar_var #CHF/kWh
öl_verbrauch = 900000 #kwh
öl_tarif = 0.13 * Tar_var #CHF/kWh
Zins_WACC = 0.0413 #2024
#Zeiten für Hochtarif
zeit_hochtarif_woche_start = datetime.datetime.strptime("06:00:00", "%H:%M:%S")
zeit_hochtarif_woche_ende = datetime.datetime.strptime("20:00:00", "%H:%M:%S")
zeit_hochtarif_samstag_start = datetime.datetime.strptime("06:00:00", "%H:%M:%S")
zeit_hochtarif_sonntag_ende = datetime.datetime.strptime("13:00:00", "%H:%M:%S")

# Dimension PV-Anlage
PV_neu = dach_süd * 509 + dach_ostwest * 1206 + fassade_süd * 299 + fassade_west * 213 + fassade_nord * 299 + fassade_ost * 231 + carport * 300
PV_Dach = dach_süd * 509 + dach_ostwest * 1206
PV_Fassade = fassade_süd * 299 + fassade_west * 213 + fassade_nord * 299 + fassade_ost * 231
PV_Carport = carport * 300
output_excel_A = "Analyse_2022_PV_"+str(PV_neu)+"kWp.xlsx"

#Mobilität
#best_LKW
lkw_inv =  590812.35
lkw_betriebskosten =   36470.40
lkw_Energiekosten = 36684.02 /1.93*(1.93-0.8)* Tar_var
lkw_steuern =  36684.02 - lkw_Energiekosten
lkw_treibstoff =  lkw_Energiekosten + lkw_steuern
lkw_strecke = r_LKW1["Km gefahren"].sum() + r_LKW2["Km gefahren"].sum() + r_LKW3["Km gefahren"].sum()
N_lkw = 20 #Lebensdauer 800'000km max, 40'000 km pro jahr
Z_lkw = Zins_WACC #Zinssatz
A_lkw = (Z_lkw*(1+Z_lkw)**N_lkw)/((1+Z_lkw)**N_lkw-1) #Annuität
JahreskostenLKW_var = lkw_inv*(A_lkw)+lkw_betriebskosten+lkw_treibstoff
jährliche_Ausgaben_LKW = lkw_betriebskosten + lkw_treibstoff +lkw_inv * (Z_lkw / 2)


# ELKW
elkw_bax_batteriepaket = 42 #kwh
elkw1_S = 126 #kWh 3 Pakete
elkw2_S = 126 #kWh 5 Pakete
elkw3_S = 126 #kWh
Verbrauch = 0.63 #kWh/km nach Bax
ladeleistungDC = 90 #kW
schnellladung = 100 #kW an schnellladesäulen
reserve_akku = 0 # 0%
ELKW_SOC_limit_Ruhezeit = 0.1 #Ladelimite von Netz ausserhalb der Betriebszeiten (Status = 0)
Grenze_SOC_Raststätte_laden = 0.05
laden_30min_raststätte = 50 #kWh mit 100 kW laden

elkw_inv_batt = 15000 * 3 #Angabe bax
elkw_inv =   190000 - elkw_inv_batt #Totale Kosten abzüglich batterie = rest vom fahrzeug (angabe bax)
elkw_betriebskosten = 550*12 #Wartungsverrtag bsp Renault, ein drittel abziehen
N_elkw = 20 #Lebensdauer 800'000km max, 40'000 km pro jahr
N_batt = 10
Z_elkw = Zins_WACC #Zinssatz
A_elkw = (Z_elkw*(1+Z_elkw)**N_elkw)/((1+Z_elkw)**N_elkw-1) #Annuität
A_elkw_batt = (Z_elkw*(1+Z_elkw)**N_batt)/((1+Z_elkw)**N_batt-1)
JahreskostenELKW_var = elkw_inv*A_elkw + elkw_betriebskosten + elkw_inv_batt*A_elkw_batt

dc_ladestation_inv = 50000 + 10000 + 700
dc_ladestation_unterhalt = 1000 + 14.90 *12
N_dc_lad = 13
A_dc_ladestation = (Z_elkw*(1+Z_elkw)**N_dc_lad)/((1+Z_elkw)**N_dc_lad-1) #lebensdauer wie batterie 10 jahre
Jahreskosten_dc_ladestation = (dc_ladestation_inv*A_dc_ladestation + dc_ladestation_unterhalt)

# WELKW
Kapazität_Wechselbatterie = 42 #kWh
welkw1_S = 126 #kWh
welkw2_S = 126 #kWh
welkw3_S = 126 #kWh
Kosten_Wechselbatterie = 20 + Kapazität_Wechselbatterie * tarif_schnellladen # Grundgebühr plus energiekosten kwh CHF
SOC_fuer_wechsel = 0.1

welkw_inv = 190000 - elkw_inv_batt #Totale Kosten abzüglich batterie = rest vom fahrzeug // Gleich wie bei elkw
welkw_betriebskosten = 550*12 #Wartungsverrtag bsp Renault
welkw_miete_wechselbatterie = (elkw_inv_batt/3/10)*1.5 * 3 #aufrundung der batteriekosten elkw auf jahr gerechnet // multipliziert mit anzahl
welkw_miete_wechselbatterie = 300*1.26*12 #neue Annahme von Nio, 300 CHF/100kWh/m
A_elkw = (Z_elkw*(1+Z_elkw)**N_elkw)/((1+Z_elkw)**N_elkw-1) #Annuität
JahreskostenWELKW_var = welkw_inv*A_elkw + welkw_betriebskosten + welkw_miete_wechselbatterie

# EPKW
N_Ladestationen = 10
durchsch_kapazität = 68 #durchschnittlich nutzbare kapazität epkw
durchsch_Reichweite = 350
durchsch_verbrauch = 0.2 #kWh/km
epkw_ladekapazitaet = 10 #kWh, 50km, wie viel laden pro tag!
ladeleistung_stationen = 22 #kW
reduktion_ladeleistung = 0.25 # über die 2h nur mit 25% laden
epkw_wochenstart_kWh = 0.5

ac_ladestation_inv = (1200 + 1000 + 700) *N_Ladestationen
ac_ladestation_unterhalt = (500 + 14.90 *12) * N_Ladestationen
N_ac_lad = 9
A_ac_ladestation = (Z_elkw*(1+Z_elkw)**N_ac_lad)/((1+Z_elkw)**N_ac_lad-1) #lebensdauer wie batterie 10 jahre
Jahreskosten_ac_ladestation = ac_ladestation_inv*A_ac_ladestation + ac_ladestation_unterhalt

#Speicher Parameter
Eigenverbrauch = 1    ## 0, 1, 2, 3
# 0 nur Peak-Shaving und keine EVerhöhung
# 1 nur wochenende EVerhöhung
# 2 auch tiefe lasten EVerhöhung (eigenverbrauchsgrenze) zb. vor und nach rückspeisung
# 3 nur Eigenverbrauch, heisst kein Netzladen
# Sicherheit laden
Faktor_Grenze = 0.01  # Sicherheitsfaktor (bsp 0.1 = 10% Sicherheit, der angestrebte max. Peak wird um 10% der Lastreduktion erhöht)
# Grenze Speicher laden von Netz im Lastprofil [Faktor für Bezug auf Spitzen Grenze, Speicher abhängig]
Faktor_laden = 1
ladeverlust = 0.1
# Wenn last unter dieser grenze Speicher entladen, kW, sollte so gewählt werden, dass nur vor und nach PV Rückspeisung entladen wird, nicht bei tiefer Grundlast
Eigenverbrauch_grenze = 50
#speicher Kostendegression bsp 0.2 = 20% Kostensenkung
Faktor_vergünstigung = 0

#Business-Case PV
Vergleich_Fassade = 70 # CHF/m2
N_PV = 25 #Lebensdauer
Z_PV = Zins_WACC #Zinssatz
A_PV = (Z_PV*(1+Z_PV)**N_PV)/((1+Z_PV)**N_PV-1) #Annuität
BK_PV =  0.01 #Betriebskosten % Invkosten
Faktor_Fassade = 1.5 #Mehrkosten für Installation an Fassade (Montage, Verkabelung) nach Axpo 50%
Capex_PV = ((5523/(PV_Dach)**0.4862)+156.2*np.exp(-0.2321*(PV_Dach))+578.4)*(PV_Dach)  #Investitionskosten nach Marktbeobachtungsstudie 2020
if PV_Fassade > 0:
    Capex_PV = ((5523 / (PV_Dach) ** 0.4862) + 156.2 * np.exp(-0.2321 * (PV_Dach)) + 578.4) * (PV_Dach) + ((5523 / (PV_Fassade) ** 0.4862) + 156.2 * np.exp(-0.2321 * (PV_Fassade)) + 578.4) * (PV_Fassade) * Faktor_Fassade
    Einsparung_Blech = (fassade_süd*1388 + fassade_west*988 + fassade_nord*1388 + fassade_ost*1075) * Vergleich_Fassade
    Capex_PV = Capex_PV - Einsparung_Blech
if PV_Carport > 0:
    Capex_PV = Capex_PV + 778752
JahreskostenPV_var = Capex_PV*(A_PV+BK_PV)
jährliche_Ausgaben_PV = Capex_PV * (BK_PV + Z_PV / 2)
EIV = dach_süd *157470 +dach_ostwest *331619 +fassade_süd *116629 +fassade_west *84810 +fassade_nord *116629 +fassade_ost *91469 + carport * 86999.85

#Business-Case Speicher
N_BS = 15
Z_BS = Zins_WACC
A_BS = (Z_BS*(1+Z_BS)**N_BS)/((1+Z_BS)**N_BS-1)

if __name__ == '__main__':
    print_start('Energiekonzept Lupfig')

from Sim_V2 import data_S, BILANZ_TOTAL
if Sim_LG ==1:
    from Sim_V2 import BILANZ_LG
if PV_neu >0:
    from Sim_V2 import LOAD_RED_PV, BILANZ_PV
if ELKW_Sim >=1:
    from Sim_V2 import INFO_ELKW, BILANZ_ELKW
if WELKW_Sim >=1:
    from Sim_V2 import INFO_WELKW, BILANZ_WELKW
if EPKW_Sim ==1:
    from Sim_V2 import INFO_EPKW, BILANZ_EPKW
if len(Speicher) >1:
    from Sim_V2 import new_monthly_limit, PROFIL_SPEICHER, max_limit_source, INFO_SPEICHER, MAX_MONTH, LOAD_RED, BILANZ_BATTERIESPEICHER

Parameters_data = {
    'LG_name': LG_name,
    'Leistungsspitzenpreis [CHF/kW]': leistungsspitzenpreis,
    'Tarifvariation [%]': "+"+str(round((Tar_var-1)*100)),
    'Hochtarif [CHF/kWh]': hochtarif,
    'Niedertarif [CHF/kWh]': niedertarif,
    'Rückspeisungstarif [CHF/kWh]': rueckspeisungstarif,
    'Öl_verbrauch [kWh/a]': round(öl_verbrauch),
    'Öl_tarif [CHF/kWh]': öl_tarif,
    'PV kWp [kW]': PV_neu
}
if len(Speicher)>1:
    Parameters_data["Speicherkapazitäten [kWh]"] = Speicher
    Parameters_data["Sicherheitsfaktor Speicher"] = Faktor_Grenze
reshaped_para = {
    'Bezeichnung': [key for key in Parameters_data.keys()],
    'Wert': [value for value in Parameters_data.values()]
}
PARAMETERS = pd.DataFrame(reshaped_para)

Diesel_data = {
    'Investitionskosten 3 LKWs [CHF]': round(lkw_inv),
    'Unterhaltskosten [CHF/a]': round(lkw_betriebskosten),
    "Energiekosten [CHF/a]" : round(lkw_Energiekosten),
    "Steuern [CHF/a]" : round(lkw_steuern),
    'Total Strecke [km/a]': round(lkw_strecke),
    "Lebensdauer [a]" : round(N_lkw),
    'Jahreskosten [CHF/a]': round(JahreskostenLKW_var),
}
reshaped_diesel = {
    'Bezeichnung': [key for key in Diesel_data.keys()],
    'Wert': [value for value in Diesel_data.values()]
}
BILANZ_DIESEL = pd.DataFrame(reshaped_diesel)

if excel:
    with pd.ExcelWriter(output_excel_S) as writer:
        PARAMETERS.to_excel(writer, sheet_name='Parameter')
        data_S.to_excel(writer, sheet_name='Lastgang')
        BILANZ_LG.to_excel(writer, sheet_name='Lastgang_Bilanz')
        #LOAD_RED_PV.to_excel(writer, sheet_name='load_red_PV')
        BILANZ_PV.to_excel(writer, sheet_name='PV_Bilanz')
        # new_monthly_limit.to_excel(writer, sheet_name='new_limit')
        BILANZ_DIESEL.to_excel(writer, sheet_name='Dlkw_Bilanz')
        if ELKW_Sim > 1:
            #INFO_ELKW.to_excel(writer, sheet_name='Elkw_sim')
            BILANZ_ELKW.to_excel(writer, sheet_name='Elkw_Bilanz')
        if WELKW_Sim > 1:
            # INFO_WELKW.to_excel(writer, sheet_name='WElkw_sim')
            BILANZ_WELKW.to_excel(writer, sheet_name='WElkw_Bilanz')
        if EPKW_Sim>0:
            #INFO_EPKW.to_excel(writer, sheet_name='Epkw_sim')
            BILANZ_EPKW.to_excel(writer, sheet_name='Epkw_Bilanz')
        if len(Speicher)>1:
            #PROFIL_SPEICHER.to_excel(writer, sheet_name='Speicher_sim')
            #max_limit_source.to_excel(writer, sheet_name='Peak_Day_PV')
            #INFO_SPEICHER.to_excel(writer, sheet_name='Info_S')
            #MAX_MONTH.to_excel(writer, sheet_name='max_month_S')
            #LOAD_RED.to_excel(writer, sheet_name='load_red_S')
            BILANZ_BATTERIESPEICHER.to_excel(writer, sheet_name='Batteriespeicher_Bilanz')
        BILANZ_TOTAL.to_excel(writer, sheet_name='Total_Bilanz')

if plot:
    #plot elkw
    if ELKW_Sim >1:
        for elkw in range(1,ELKW_Sim+1):
            if elkw ==1:
                time_windows = [('2022-04-27 00:00:00', '2022-04-28 00.00:00'),
                                ('2022-05-23 00:00:00', '2022-05-30 00:00:00'),
                                ('2022-11-14 00:00:00', '2022-11-21 00:00:00')]
            else:
                time_windows = [('2022-05-23 00:00:00', '2022-05-28 00:00:00'),
                                ('2022-11-14 00:00:00', '2022-11-21 00:00:00')]
            for start_time, end_time in time_windows:
                subset_lkw = INFO_ELKW.loc[(INFO_ELKW['Time'] >= start_time) & (INFO_ELKW['Time'] <= end_time)]
                subset_lg = data_S.loc[(INFO_ELKW['Time'] >= start_time) & (INFO_ELKW['Time'] <= end_time)]

                sns.set(rc={'figure.dpi': 300})
                sns.set_style("whitegrid")
                fig, ax1 = plt.subplots(figsize=(10, 6))

                sns.lineplot(x="Time", y="kW_PV_S", data=subset_lg, ax=ax1, linewidth=2.5, color='orange',
                             label="Last ohne E-LKW")
                sns.lineplot(x="Time", y="kW_RS_S", data=subset_lg, ax=ax1, linewidth=2.5, color='gold',
                             label="Rücksp ohne E-LKW")
                sns.lineplot(x="Time", y="Last_ELKW"+str(elkw), data=subset_lkw, ax=ax1, linewidth=2.5, color='green',
                             label="Last mit E-LKW")
                sns.lineplot(x="Time", y="Ruecksp_ELKW"+str(elkw), data=subset_lkw, ax=ax1, linewidth=2.5, color='yellowgreen',
                             label="Rücksp mit E-LKW")

                ax1.set_ylim(0, 1000)
                ax1.set_ylabel('Lastgang [kW]')

                ax2 = ax1.twinx()
                sns.lineplot(x="Time", y="ELKW"+str(elkw)+"_SOC", data=subset_lkw, ax=ax2, linewidth=2.5, color='deepskyblue', label="SOC")
                ax2.set_ylim(0, 1)
                ax2.set_ylabel('SOC')

                # Set the x-axis limits to match the start and end time
                ax1.set_xlim(pd.Timestamp(start_time), pd.Timestamp(end_time))

                # Set the major ticks to be at every 12 hours and format the tick labels
                hours = mdates.HourLocator(interval=12)
                fmt = mdates.DateFormatter('%H:%M\n%d.%m')
                ax1.xaxis.set_major_locator(hours)
                ax1.xaxis.set_major_formatter(fmt)

                # Set the first tick to be at the start time
                ax1.xaxis.set_tick_params(which='major', pad=15)
                first_tick = pd.Timestamp(start_time).floor('D')
                ax1.set_xticks(pd.date_range(first_tick, pd.Timestamp(end_time), freq='12H'))

                ax1.legend_.remove()
                ax2.legend_.remove()
                fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax2.transAxes)
                plt.title('Auswirkung E-LKW'+str(elkw))
                plt.show()

    #plot welkw
    if WELKW_Sim >1:
        for welkw in range(1,WELKW_Sim+1):
            if welkw ==1:
                time_windows = [('2022-04-27 00:00:00', '2022-04-28 00.00:00'),
                                ('2022-05-23 00:00:00', '2022-05-30 00:00:00'),
                                ('2022-11-14 00:00:00', '2022-11-21 00:00:00')]
            else:
                time_windows = [('2022-05-23 00:00:00', '2022-05-30 00:00:00'),
                                ('2022-11-14 00:00:00', '2022-11-21 00:00:00')]
            for start_time, end_time in time_windows:
                subset_lkw = INFO_WELKW.loc[(INFO_WELKW['Time'] >= start_time) & (INFO_WELKW['Time'] <= end_time)]
                subset_lg = data_S.loc[(INFO_WELKW['Time'] >= start_time) & (INFO_WELKW['Time'] <= end_time)]

                sns.set(rc={'figure.dpi': 300})
                sns.set_style("whitegrid")
                fig, ax1 = plt.subplots(figsize=(10, 6))

                sns.lineplot(x="Time", y="kW_PV_S", data=subset_lg, ax=ax1, linewidth=2.5, color='orange',
                             label="Last best.")
                sns.lineplot(x="Time", y="kW_RS_S", data=subset_lg, ax=ax1, linewidth=2.5, color='gold',
                             label="Rücksp best.")
                sns.lineplot(x="Time", y="Last_WELKW"+str(welkw), data=subset_lkw, ax=ax1, linewidth=2.5, color='green',
                             label="Last WElkw")
                sns.lineplot(x="Time", y="Ruecksp_WELKW"+str(welkw), data=subset_lkw, ax=ax1, linewidth=2.5, color='yellowgreen',
                             label="Rücksp WElkw")
                ax1.set_ylim(0, 1000)
                ax1.set_ylabel('Lastgang [kW]')

                ax2 = ax1.twinx()
                sns.lineplot(x="Time", y="WELKW"+str(welkw)+"_SOC", data=subset_lkw, ax=ax2, linewidth=2.5, color='deepskyblue', label="SOC")
                ax2.set_ylim(0, 1)
                ax2.set_ylabel('SOC')

                # Set the x-axis limits to match the start and end time
                ax1.set_xlim(pd.Timestamp(start_time), pd.Timestamp(end_time))

                # Set the major ticks to be at every 12 hours and format the tick labels
                hours = mdates.HourLocator(interval=12)
                fmt = mdates.DateFormatter('%H:%M\n%d.%m')
                ax1.xaxis.set_major_locator(hours)
                ax1.xaxis.set_major_formatter(fmt)

                # Set the first tick to be at the start time
                ax1.xaxis.set_tick_params(which='major', pad=15)
                first_tick = pd.Timestamp(start_time).floor('D')
                ax1.set_xticks(pd.date_range(first_tick, pd.Timestamp(end_time), freq='12H'))

                ax1.legend_.remove()
                ax2.legend_.remove()
                fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax2.transAxes)
                plt.title('Auswirkung WE-LKW'+str(welkw))
                plt.show()

    #plot epkw
    if EPKW_Sim ==1:
        time_windows = [('2022-05-23 00:00:00', '2022-05-30 00:00:00'),
                        ('2022-11-14 00:00:00', '2022-11-21 00:00:00')]
        for start_time, end_time in time_windows:
            subset_epkw = INFO_EPKW.loc[(INFO_EPKW['Time'] >= start_time) & (INFO_EPKW['Time'] <= end_time)]
            if ELKW_Sim ==0:
                subset_lg = data_S.loc[(INFO_EPKW['Time'] >= start_time) & (INFO_EPKW['Time'] <= end_time)]
            else:
                subset_lkw = INFO_ELKW.loc[(INFO_ELKW['Time'] >= start_time) & (INFO_ELKW['Time'] <= end_time)]
            sns.set(rc={'figure.dpi': 300})
            sns.set_style("whitegrid")
            fig, ax1 = plt.subplots(figsize=(10, 6))

            if ELKW_Sim ==0:
                sns.lineplot(x="Time", y="kW_PV_S", data=subset_lg, ax=ax1, linewidth=2.5, color='orange',
                             label="Last ohne EPKW")
                sns.lineplot(x="Time", y="kW_RS_S", data=subset_lg, ax=ax1, linewidth=2.5, color='gold',
                             label="Rücksp ohne EPKW")
            else:
                sns.lineplot(x="Time", y="Last_ELKW" + str(elkw), data=subset_lkw, ax=ax1, linewidth=2.5, color='orange',
                             label="Last ohne EPKW")
                sns.lineplot(x="Time", y="Ruecksp_ELKW" + str(elkw), data=subset_lkw, ax=ax1, linewidth=2.5, color='gold',
                             label="Rücksp ohne EPKW")
            sns.lineplot(x="Time", y="Last_EPKW", data=subset_epkw, ax=ax1, linewidth=2.5, color='green',
                         label="Last mit EPKW")
            sns.lineplot(x="Time", y="Ruecksp_EPKW", data=subset_epkw, ax=ax1, linewidth=2.5, color='yellowgreen',
                         label="Rücksp mit EPKW")

            ax1.set_ylim(0, 1000)
            ax1.set_ylabel('Lastgang [kW]')

            ax2 = ax1.twinx()
            sns.lineplot(x="Time", y="EPKW_SOC", data=subset_epkw, ax=ax2, linewidth=2.5, color='deepskyblue', label="SOC")
            ax2.set_ylim(0, 1)
            ax2.set_ylabel('SOC')

            # Set the x-axis limits to match the start and end time
            ax1.set_xlim(pd.Timestamp(start_time), pd.Timestamp(end_time))

            # Set the major ticks to be at every 12 hours and format the tick labels
            hours = mdates.HourLocator(interval=12)
            fmt = mdates.DateFormatter('%H:%M\n%d.%m')
            ax1.xaxis.set_major_locator(hours)
            ax1.xaxis.set_major_formatter(fmt)

            # Set the first tick to be at the start time
            ax1.xaxis.set_tick_params(which='major', pad=15)
            first_tick = pd.Timestamp(start_time).floor('D')
            ax1.set_xticks(pd.date_range(first_tick, pd.Timestamp(end_time), freq='12H'))
            ax1.legend_.remove()
            ax2.legend_.remove()
            fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)
            plt.title('Auswirkung E-PKW')
            plt.show()

    #plot Batteriespeicher
    if len(Speicher)>1:
        for S in Speicher:
            if S>0:
                time_windows = [('2022-07-11 00:00:00', '2022-07-12 00:00:00'),
                                ('2022-01-10 00:00:00', '2022-01-11 00:00:00'),
                                ('2022-03-02 00:00:00', '2022-03-04 00:00:00'),
                                ('2022-09-01 00:00:00', '2022-09-02 00:00:00')]
                for start_time, end_time in time_windows:
                    subset_Speicher = PROFIL_SPEICHER.loc[(PROFIL_SPEICHER['Time'] >= start_time) & (PROFIL_SPEICHER['Time'] <= end_time)]
                    subset_SOC = INFO_SPEICHER.loc[(INFO_SPEICHER['Time'] >= start_time) & (INFO_SPEICHER['Time'] <= end_time)]

                    sns.set(rc={'figure.dpi': 300})
                    sns.set_style("whitegrid")
                    fig, ax1 = plt.subplots(figsize=(10, 6))

                    sns.lineplot(x="Time", y="Last_0", data=subset_Speicher, ax=ax1, linewidth=2.5, color='orange',
                                 label="Last ohne Speicher")
                    sns.lineplot(x="Time", y="Ruecksp_0", data=subset_Speicher, ax=ax1, linewidth=2.5, color='gold',
                                 label="Rücksp ohne Speicher")
                    sns.lineplot(x="Time", y="Last_"+str(S), data=subset_Speicher, ax=ax1, linewidth=2.5, color='green',
                                 label="Last mit Speicher")
                    sns.lineplot(x="Time", y="Ruecksp_"+str(S), data=subset_Speicher, ax=ax1, linewidth=2.5, color='yellowgreen',
                                 label="Rücksp mit Speicher")
                    ax1.set_ylim(0, 1000)
                    ax1.set_ylabel('Lastgang [kW]')

                    ax2 = ax1.twinx()
                    sns.lineplot(x="Time", y=str(S)+"_SOC", data=subset_SOC, ax=ax2, linewidth=2.5, color='deepskyblue', label="SOC")
                    ax2.set_ylim(0, 1)
                    ax2.set_ylabel('SOC')

                    # Set the x-axis limits to match the start and end time
                    ax1.set_xlim(pd.Timestamp(start_time), pd.Timestamp(end_time))

                    # Set the major ticks to be at every 12 hours and format the tick labels
                    hours = mdates.HourLocator(interval=12)
                    fmt = mdates.DateFormatter('%H:%M\n%d.%m')
                    ax1.xaxis.set_major_locator(hours)
                    ax1.xaxis.set_major_formatter(fmt)

                    # Set the first tick to be at the start time
                    ax1.xaxis.set_tick_params(which='major', pad=15)
                    first_tick = pd.Timestamp(start_time).floor('D')
                    ax1.set_xticks(pd.date_range(first_tick, pd.Timestamp(end_time), freq='12H'))

                    ax1.legend_.remove()
                    ax2.legend_.remove()
                    fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax2.transAxes)
                    plt.title('Auswirkung Batteriespeicher'+str(S))
                    plt.show()

    #plot PV
    if len(Speicher)>0:
        time_windows = [('2022-05-23 00:00:00', '2022-05-30 00:00:00'),
                        ('2022-07-18 00:00:00', '2022-07-25 00:00:00'),
                        ('2022-08-08 00:00:00', '2022-08-15 00:00:00'),
                        ('2022-12-12 00:00:00', '2022-12-19 00:00:00')]
        for start_time, end_time in time_windows:
            subset_lg = data_S.loc[(data_S['Time'] >= start_time) & (data_S['Time'] <= end_time)]

            sns.set(rc={'figure.dpi': 300})
            sns.set_style("whitegrid")
            fig, ax1 = plt.subplots(figsize=(10, 6))

            sns.lineplot(x="Time", y="kW_Last", data=subset_lg, ax=ax1, linewidth=2.5, color='orange',
                         label="Last ohne PV")
            sns.lineplot(x="Time", y="kW_PV_S", data=subset_lg, ax=ax1, linewidth=2.5, color='green',
                         label="Last mit PV")
            sns.lineplot(x="Time", y="kW_RS_S", data=subset_lg, ax=ax1, linewidth=2.5, color='yellowgreen',
                         label="Rücksp mit PV")
            ax1.set_ylim(0, 1000)
            ax1.set_ylabel('Lastgang [kW]')

            # Set the x-axis limits to match the start and end time
            ax1.set_xlim(pd.Timestamp(start_time), pd.Timestamp(end_time))

            # Set the major ticks to be at every 12 hours and format the tick labels
            hours = mdates.HourLocator(interval=12)
            fmt = mdates.DateFormatter('%H:%M\n%d.%m')
            ax1.xaxis.set_major_locator(hours)
            ax1.xaxis.set_major_formatter(fmt)

            # Set the first tick to be at the start time
            ax1.xaxis.set_tick_params(which='major', pad=15)
            first_tick = pd.Timestamp(start_time).floor('D')
            ax1.set_xticks(pd.date_range(first_tick, pd.Timestamp(end_time), freq='12H'))
            ax1.legend_.remove()
            fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)
            plt.title('Auswirkung PV '+str(PV_neu)+'kWp')
            plt.show()

end_timer = time.perf_counter()
running_time_min = round((end_timer-start_timer)/60,2)
print("Programm fertig: Laufzeit = "+str(running_time_min)+" MIN")
quit()

Start Simulation Energiekonzept Lupfig
Lastgang lesen und PV Eigenverbrauch berechnen
Mobilität einlesen
Simulation Running
Analyse für monatliche Spitze
Analyse für monatliche Spitze
Simulation Done!
Programm fertig: Laufzeit = 5.55 MIN
